In [217]:
import pandas as pd
import numpy as np
import pandasql as ps
import matplotlib.pyplot as plt


from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

np.random.seed(0)

In [218]:
#load dataframe from csv
merged_sales = pd.read_csv("data/zip/merged_sales_stats.csv", delimiter='	')

#print dataframe shape
shape = merged_sales.shape
print('\nDataFrame Shape :', shape)

print(merged_sales.columns)


DataFrame Shape : (137166, 23)
Index(['Unnamed: 0', 'Zip Code', 'Month of Period End', 'Median Sale Price',
       'Median Sale Price MoM ', 'Median Sale Price YoY ', 'Homes Sold',
       'Homes Sold MoM ', 'Homes Sold YoY ', 'New Listings',
       'New Listings MoM ', 'New Listings YoY ', 'Inventory', 'Inventory MoM ',
       ' Inventory YoY ', 'Days on Market', 'Days on Market MoM',
       'Days on Market YoY', 'Average Sale To List',
       'Average Sale To List MoM ', 'Average Sale To List YoY ',
       'record_month', 'PSSF'],
      dtype='object')


The merged_sales_statistics file has all the statistics as columns and zip codes and dates as the rows. I tried to transpose these but it got weird. So let's get some stuff from the merged_sales:

In [219]:


#load dataframe from csv
merged_bedroom_data = pd.read_csv("data/zip/merged_bedrooms.csv", delimiter='	')
print(merged_bedroom_data.shape)
print(merged_bedroom_data.columns)


(1688, 682)
Index(['Unnamed: 0', 'Zip Code', 'June 2009 1bd', 'July 2009 1bd',
       'August 2009 1bd', 'September 2009 1bd', 'October 2009 1bd',
       'November 2009 1bd', 'December 2009 1bd', 'January 2010 1bd',
       ...
       'December 2019 5bd', 'January 2020 5bd', 'February 2020 5bd',
       'March 2020 5bd', 'April 2020 5bd', 'May 2020 5bd', 'June 2020 5bd',
       'July 2020 5bd', 'August 2020 5bd', 'September 2020 5bd'],
      dtype='object', length=682)


In [220]:
''' 
now for our demographic info, it's from 2019 so not super up to date, but pretty close and there's a lot of it
'''
demographics = pd.read_csv("data/zip/ca_demographics.csv", delimiter='	')
zip_to_county = pd.read_csv("data/zip/zip_to_county.csv", delimiter='	')

print(demographics.shape) # a lot of blank columns


print(demographics["SE_A18003_001"])

(41, 207)
0     29.100000
1     31.900000
2     30.700001
3     37.599998
4     33.000000
5     36.099998
6     30.799999
7     33.700001
8     29.500000
9     29.100000
10    33.299999
11    30.299999
12    32.099998
13    31.799999
14    27.299999
15    32.799999
16    29.900000
17    42.599998
18    32.099998
19    30.799999
20    33.200001
21    32.400002
22    33.000000
23    32.700001
24    22.299999
25    31.500000
26    33.099998
27    27.600000
28    33.000000
29    27.600000
30    32.200001
31    27.200001
32    30.600000
33    31.500000
34    31.700001
35    33.500000
36    26.700001
37    29.400000
38    33.200001
39    31.000000
40    30.600000
Name: SE_A18003_001, dtype: float64


Most of the column names are gibberish, so we'll need to use the [ca_demographics_key.txt](data/zip/ca_demographics_key.txt) to make sense of them.

In [221]:
zipdemo = pd.read_csv("data/zip/ca_zip_demograhics.csv", delimiter='	')
print(zipdemo.shape)

(1763, 217)


There's a lot to look at in that dataset, but certainly all the key indicators about the zip should give us at least some picture of what the housing market there might look like.

##### Condensing Sale Data to Year

In [222]:
#remove month from "Month of Period End"
only_year_query = """SELECT "Zip Code" as zip_code, 
                    SUBSTR("Month of Period End" ,-4) as year,
                    "Median Sale Price" as med_sale_price,
                    "Homes Sold" as home_sold,
                    "New Listings" as new_listings,
                    "Inventory" as inventory,
                    "Days on Market" as days_on_market,
                    "Average Sale To List" as avg_sale_to_list,
                    PSSF as ppsf
                    FROM merged_sales"""
only_year = ps.sqldf(only_year_query, locals())

#group by year
cond_year_query = """SELECT zip_code, 
                            year,
                            AVG(med_sale_price) as med_sale_price,
                            SUM(home_sold) as homes_sold,
                            SUM(new_listings) as new_listings,
                            AVG(inventory) as inventory,
                            AVG(days_on_market) as days_on_market,
                            AVG(avg_sale_to_list) as avg_sale_to_list,
                            AVG(ppsf) as ppsf
                    FROM only_year
                    GROUP BY year, zip_code"""
cond_year = ps.sqldf(cond_year_query, locals())

#filtering to 2018
year_18_query = """SELECT *
                    FROM cond_year
                    WHERE year = '2018' """
year_18 = ps.sqldf(year_18_query, locals())

### 2018 Sales and Demographic data

In [226]:
#zip code = Geo_ZCTA5

sales_demo_query = """SELECT year_18.*, zipdemo.*
                    FROM year_18
                    INNER JOIN zipdemo on zip_code = Geo_ZCTA5
                    """
sales_demo = ps.sqldf(sales_demo_query, locals())
sales_demo

,zip_code,year,med_sale_price,homes_sold,new_listings,inventory,days_on_market,avg_sale_to_list,ppsf,Geo_FIPS,...,SE_A08002B_005,SE_A08002B_006,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008
0,90001,2018,4.228333e+05,511,678.0,44.333333,43.166667,100.125000,322.500000,90090001,...,144,118,13815,1834,2083,2594,2513,2045,1330,1416
1,90002,2018,3.825000e+05,964,1178.0,74.666667,39.916667,99.916667,323.750000,90090002,...,45,124,12706,2096,2232,2020,2266,1719,895,1478
2,90003,2018,4.076667e+05,962,1222.0,96.000000,45.416667,99.933333,314.166667,90090003,...,193,129,17127,2594,2479,2966,3345,2757,1476,1510
3,90004,2018,1.286500e+06,747,1117.0,85.833333,33.833333,99.233333,630.916667,90090004,...,483,463,21971,6379,6385,3842,2854,1539,721,251
4,90005,2018,7.710000e+05,314,446.0,30.666667,30.250000,99.225000,523.000000,90090005,...,431,380,16442,6298,4939,2183,1794,832,293,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,96146,2018,5.807500e+05,249,368.0,62.666667,163.333333,95.691667,486.333333,96196146,...,0,9,454,152,215,31,24,32,0,0
1333,96148,2018,6.440833e+05,155,156.0,14.583333,90.500000,96.950000,413.666667,96196148,...,24,0,210,55,62,6,50,26,0,11
1334,96150,2018,4.536667e+05,2333,3039.0,246.333333,43.750000,97.008333,307.666667,96196150,...,461,24,11536,3645,4315,1666,1221,427,202,60
1335,96155,2018,2.340000e+05,8,NaN,NaN,104.000000,91.333333,145.666667,96196155,...,0,0,0,0,0,0,0,0,0,0


In [231]:

#create avg_sale_to_list groups (split into 5)
ntile_DoM_5_query = """SELECT *, NTILE(5) OVER (
                            ORDER BY days_on_market ASC) as DoM_group      
                        FROM sales_demo"""
ntile_DoM_5 = ps.sqldf(ntile_DoM_5_query, locals())


#see group maximums
minmax_DoM_5_query = """SELECT DoM_group, COUNT(zip_code), MAX(days_on_market)
                        FROM ntile_DoM
                        GROUP BY DoM_group"""
minmax_DoM_5 = ps.sqldf(minmax_DoM_5_query, locals())
minmax_DoM_5



#Delete non-numeric and unused columns.
del_col = ['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_QName', 'Geo_STUSAB', 'Geo_SUMLEV', 
           'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 
          'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY', 'Geo_COUSUB', 'Geo_PLACE', 'Geo_PLACESE', 
          'Geo_TRACT', 'Geo_BLKGRP', 'Geo_CONCIT', 'Geo_AIANHH', 'Geo_AIANHHFP', 'Geo_AIHHTLI', 
          'Geo_AITSCE', 'Geo_AITS', 'Geo_ANRC', 'Geo_CBSA', 'Geo_CSA', 'Geo_METDIV', 'Geo_MACC', 
          'Geo_MEMI', 'Geo_NECTA', 'Geo_CNECTA', 'Geo_NECTADIV', 'Geo_UA', 'Geo_UACP', 'Geo_CDCURR', 
          'Geo_SLDU', 'Geo_SLDL', 'Geo_VTD', 'Geo_ZCTA3', 'Geo_SUBMCD', 'Geo_SDELM', 'Geo_SDSEC', 
          'Geo_SDUNI', 'Geo_UR', 'Geo_PCI', 'Geo_TAZ', 'Geo_UGA', 'Geo_BTTR', 'Geo_BTBG', 
          'Geo_PUMA5', 'Geo_PUMA1', 'year','Geo_ZCTA5', 'avg_sale_to_list']
for i in del_col:
    del ntile_DoM_5[i]

### Building the Model

In [234]:


# Set variables to hold test and training data.

ntile_5_noDOM = ntile_DoM_5.drop(columns=["days_on_market"])

# Shuffle data frame.
shuffled_5 = ntile_5_noDOM.sample(frac=1).reset_index(drop=True)

# Remove blanks and NaNs
shuffled_5.replace('', np.nan)
shuffled_5 = shuffled_5.dropna(axis=0, how='any')


In [235]:

# Split into data and labels.
labels_5 = shuffled_5["DoM_group"]
del shuffled_5["DoM_group"]
zip_codes_5 = shuffled_5["zip_code"]
del shuffled_5["zip_code"]
shuffled_5 = shuffled_5.apply(lambda x: pd.to_numeric(x))

# Create 80% split point.
split = int(len(shuffled_5)*0.8//1)

# Store in variables.
train_data_5 = shuffled_5[:split]
train_labels_5 = labels_5[:split]
test_data_5 = shuffled_5[split+1:]
test_labels_5 = labels_5[split+1:]

In [236]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# we want to normalize all of our data if possible
sc = StandardScaler()
X_train = sc.fit_transform(train_data_5)
X_test = sc.transform(test_data_5)

#now perform PCA
pca = PCA(n_components=20)
#pca.fit(X_train)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(pca.explained_variance_ratio_)
print(" percent of variance captured:: ", np.sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.41209654 0.1536392  0.10224103 0.04995462 0.02884485 0.02170934
 0.01845116 0.01808665 0.0160347  0.0144235  0.01185792 0.00956218
 0.00877921 0.00855299 0.00782783 0.0068305  0.00566613 0.00545206
 0.00537864 0.00485091]
 percent of variance captured::  0.910239952868229
[154.0874501   94.08465663  76.75035303  53.64825541  40.76635882
  35.36641605  32.60465838  32.28098998  30.39472669  28.82724097
  26.13797732  23.47179386  22.49030802  22.19865801  21.23677073
  19.83782582  18.06804659  17.72344916  17.6037027   16.71780176]


Let's look at a Random Forest

In [237]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=32, random_state=0)
classifier.fit(X_train, train_labels)

# Predicting the Test set results
predicted_labels = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels != predicted_labels)
accuracy = (len(test_labels) - num_wrong) / len(test_labels)
print("Accuracy of our PCA model w/Random Forest and no days on market : ", accuracy)

Accuracy of our PCA model w/Random Forest and no days on market :  0.10465116279069768


Maybe XGBoost will give us something better

In [238]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

model = XGBClassifier()
model.fit(train_data, train_labels)
# make predictions for test data and evaluate
y_pred = model.predict(test_data)

print(y_pred)

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels != y_pred)
accuracy = (len(test_labels) - num_wrong) / len(test_labels)
print(accuracy)

# accuracy = accuracy_score(test_data, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold

thresholds = np.sort(model.feature_importances_)

for thresh in thresholds:
    if(thresh > 0.00001):
        # select features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(train_data)
        # train model
        selection_model = XGBClassifier()
        selection_model.fit(select_X_train, train_labels)
        # eval model
        select_X_test = selection.transform(test_data)
        y_pred = selection_model.predict(select_X_test)
        #predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(test_labels, y_pred)
        print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

[1 1 1 1 5 2 7 4 6 5 7 7 8 9 5 2 6 2 7 2 7 5 6 2 2 1 2 1 7 8 2 5 8 3 6 5 1
 7 8 2 7 2 5 1 7 3 1 9 5 4 8 4 6 8 1 8 5 4 5 5 2 1 8 5 2 5 1 5 3 8 7 5 9 5
 7 6 4 5 4 3 3 5 4 6 1 1]
0.4069767441860465
Thresh=0.000, n=151, Accuracy: 40.70%
Thresh=0.001, n=150, Accuracy: 41.86%
Thresh=0.001, n=149, Accuracy: 41.86%


KeyboardInterrupt: 

Making 10 bins and dropping them though doesn't seem to do too much better

In [239]:
#create avg_sale_to_list groups (split into 5)
ntile_10_DoM_query = """SELECT *, NTILE(10) OVER (
                            ORDER BY days_on_market ASC) as DoM_10_group      
                        FROM sales_demo"""
ntile_10_DoM = ps.sqldf(ntile_10_DoM_query, locals())


#see group maximums
minmax_10_DoM_query = """SELECT DoM_10_group, COUNT(zip_code), MAX(days_on_market)
                        FROM ntile_10_DoM
                        GROUP BY DoM_10_group"""
minmax_10_DoM = ps.sqldf(minmax_10_DoM_query, locals())

#Delete non-numeric and unused columns.
del_col = ['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_QName', 'Geo_STUSAB', 'Geo_SUMLEV', 
           'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 
          'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY', 'Geo_COUSUB', 'Geo_PLACE', 'Geo_PLACESE', 
          'Geo_TRACT', 'Geo_BLKGRP', 'Geo_CONCIT', 'Geo_AIANHH', 'Geo_AIANHHFP', 'Geo_AIHHTLI', 
          'Geo_AITSCE', 'Geo_AITS', 'Geo_ANRC', 'Geo_CBSA', 'Geo_CSA', 'Geo_METDIV', 'Geo_MACC', 
          'Geo_MEMI', 'Geo_NECTA', 'Geo_CNECTA', 'Geo_NECTADIV', 'Geo_UA', 'Geo_UACP', 'Geo_CDCURR', 
          'Geo_SLDU', 'Geo_SLDL', 'Geo_VTD', 'Geo_ZCTA3', 'Geo_SUBMCD', 'Geo_SDELM', 'Geo_SDSEC', 
          'Geo_SDUNI', 'Geo_UR', 'Geo_PCI', 'Geo_TAZ', 'Geo_UGA', 'Geo_BTTR', 'Geo_BTBG', 
          'Geo_PUMA5', 'Geo_PUMA1', 'year','Geo_ZCTA5', 'avg_sale_to_list']
for i in del_col:
    del ntile_10_DoM[i]


Try adding in some new columns around availability and affordability

In [241]:
ntile_10_noDOM = ntile_10_DoM.drop(columns=["days_on_market"])
ntile_10_noDOM['Availability'] = ntile_10_noDOM.apply(lambda row: row.new_listings / np.maximum(1.0, row.SE_A00002_002), axis = 1)
ntile_10_noDOM['cost_ratio'] = ntile_10_noDOM.apply(lambda row: row.med_sale_price / np.maximum(1.0, row.SE_A14024_001), axis = 1)

In [242]:
# Set variables to hold test and training data.

# Shuffle data frame.
shuffled_10 = ntile_10_noDOM.sample(frac=1).reset_index(drop=True)

# Remove blanks and NaNs
shuffled_10.replace('', np.nan)
shuffled_10 = shuffled_10.dropna(axis=0, how='any')

# Split into data and labels.
labels_10 = shuffled_10["DoM_10_group"]
del shuffled_10["DoM_10_group"]
zip_codes = shuffled_10["zip_code"]
del shuffled_10["zip_code"]
shuffled_10 = shuffled_10.apply(lambda x: pd.to_numeric(x))

# Create 70% split point.
split_10 = int(len(shuffled_10)*0.7//1)

# Store in variables.
train_data_10 = shuffled_10[:split]
train_labels_10 = labels_10[:split]
test_data_10 = shuffled_10[split+1:]
test_labels_10 = labels_10[split+1:]

train_data_10.head()

,med_sale_price,homes_sold,new_listings,inventory,ppsf,SE_A00001_001,SE_A00002_001,SE_A00002_002,SE_A00002_003,SE_A00003_001,...,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008,Availability,cost_ratio
10,3.156667e+05,711,813.0,27.583333,215.500000,21185,21185,2747.2800,7.711264,7.829637,...,7232,1601,2368,1298,997,554,248,166,0.295929,11.983853
13,6.925833e+05,1565,2056.0,124.666667,414.333333,45282,45282,6093.9050,7.430703,7.442647,...,18236,5355,6380,3029,2494,786,126,66,0.337386,13.831749
17,8.625000e+05,528,589.0,31.333333,451.750000,19035,19035,722.4431,26.348097,26.865210,...,7306,1831,2447,1269,1192,388,118,61,0.815289,16.806313
18,2.632500e+05,2640,3409.0,191.833333,184.916667,77006,77006,392.5612,196.163034,197.044800,...,21577,4127,4489,3921,3871,2657,1352,1160,8.683996,16.297282
19,1.045250e+06,965,1220.0,72.000000,518.000000,25218,25218,6945.3490,3.630919,3.645880,...,11491,4060,4264,1779,1106,214,20,48,0.175657,15.544138


In [243]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

model = XGBClassifier()
model.fit(train_data_10, train_labels_10)
# make predictions for test data and evaluate
y_pred = model.predict(test_data_10)

print(y_pred)

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels_10 != y_pred)
accuracy = (len(test_labels_10) - num_wrong) / len(test_labels_10)
print(accuracy)


[4 4 3 7 6 2 1 2 7 2 2 2 2 1 8 6 5 6 2 2 1 2 5 5 1 8 5 6 5 7 2 4 1 2 4 4 3
 1 1 1 6 8 5 6 4 7 3 4 6 7 1 2 2 4 1 6 3 6 9 3 1 2 4 6 7 2 8 7 8 3 8 3 6 4
 2 7 1 6 1 4 4 7 6 4 1 8]
0.3023255813953488


In [244]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = np.unique(y_train, axis = 0)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]
    return sample_weights


#pass labels as numpy array
weight = CreateBalancedSampleWeights(train_labels_10, 20.0)

#And then use it like this
xg = XGBClassifier(n_estimators=3000, weights = weight, max_depth=100)

xg.fit(train_data_10, train_labels_10)
# make predictions for test data and evaluate
y_pred = xg.predict(test_data_10)

print(y_pred)

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels_10 != y_pred)
accuracy = (len(test_labels_10) - num_wrong) / len(test_labels_10)
print(accuracy)

C:\Users\winbase\Anaconda3\envs\mids\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


[4 3 3 6 6 2 1 2 4 2 2 2 2 1 8 6 5 6 2 2 1 8 7 8 1 8 5 6 4 3 3 4 1 8 4 4 4
 1 1 1 6 8 5 7 4 7 3 4 6 7 1 2 2 4 1 7 3 6 9 1 1 2 5 6 7 7 8 7 9 3 8 3 8 2
 1 6 1 6 1 8 8 7 6 4 1 8]
0.3372093023255814


In [ ]:


thresholds = np.sort(model.feature_importances_)

for thresh in thresholds:
    if(thresh > 0.001):
        # select features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(train_data_10)
        # train model
        selection_model = XGBClassifier()
        selection_model.fit(select_X_train, train_labels)
        # eval model
        select_X_test = selection.transform(test_data_10)
        y_pred = selection_model.predict(select_X_test)
        #predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(test_labels_10, y_pred)
        print("Thresh=%.3f, n=%d, Accuracy: %.4f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

How can GridSearch help us find good MLP params?

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

parameters = {'solver': ['lbfgs'], 'max_iter': [100, 300, 500, 700, 1200, 2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(3, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}

train_data_scaled = preprocessing.scale(train_data_10)

clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

clf.fit(train_data_10, train_labels_10)
print(clf.score(test_data_10, test_labels_10))
print(clf.best_params_)

y_pred = clf.predict(test_data_10)

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels_10 != y_pred)
accuracy = (len(test_labels_10) - num_wrong) / len(test_labels_10)
print(accuracy)